# Nearest Restaurants in Madrid

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import the API key
from config import geoapify_key

In [2]:
# Load the restaurant ethnicities into a DataFrame
types_df = pd.read_csv("../Resources/ethnicities.csv")

# Display sample data
types_df.head()

,ethnicity
0,italian
1,chinese
2,mexican
3,japanese
4,american


In [3]:
# Set up additional columns to hold information
types_df["name"] = ""
types_df["address"] = ""
types_df["distance"] = ""

# Display sample data
types_df.head()

,ethnicity,name,address,distance
0,italian,,,
1,chinese,,,
2,mexican,,,
3,japanese,,,
4,american,,,


Find the closest restaurant of each type to the Plaza Mayor in Madrid, Spain

* Latitude: 40.415392

* Longitude: -3.7073743182788528

In [4]:
# Set the geographical coordinates
latitude = 40.415392
longitude = -3.7073743182788528

# Set the parameters for the type of search
limit = 20
radius = 1000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [5]:
# Print a message to follow up the restaurant search
print("Starting restaurant search")

# Iterate through the types_df DataFrame
for index, row in types_df.iterrows():
    
    # Get the ethnicity type from the current DataFrame's row
    ethnicity = types_df.loc[index, "ethnicity"]
    # Add the current ethnicity type to the parameters
    params["categories"] = f"catering.restaurant.{ethnicity}"
   
    # Make an API request using the params dictionary
    restaurant = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    restaurant = restaurant.json()
    
    # Grab the first restaurant from the results and store the details in the DataFrame
    try:
        types_df.loc[index, "name"] = restaurant["features"][0]["properties"]["name"]
        types_df.loc[index, "address"] = restaurant["features"][0]["properties"]["address_line2"]
        types_df.loc[index, "distance"] = int(restaurant["features"][0]["properties"]["distance"])
    except (KeyError, IndexError):
        # If no restaurant is found, set the restaurant name as "No restaurant found".
        types_df.loc[index, "name"] = "No restaurant found"
        # Set the distance column value to np.nan to allow sorting values
        types_df.loc[index, "distance"] = np.nan
        
    # Log the search results
    print(f"nearest {types_df.loc[index, 'ethnicity']} restaurant: {types_df.loc[index, 'name']}")

# Display sample data
types_df

Starting restaurant search
nearest italian restaurant: La Fojetta
nearest chinese restaurant: Tapas de Sichuan
nearest mexican restaurant: Takos al Pastor
nearest japanese restaurant: Sushi Bar


KeyboardInterrupt: 

In [ ]:
# Sort restuls by distance
types_df = types_df.sort_values(by=["distance"])

# Display the top 10 nearest restaurants
types_df.head(10)